In [3]:
import random
import time
import datetime
import pandas as pd
import csv
import pandas as pd
import lightgbm as lgb
from Intelligent_well_control.src.models.utils.save_to_csv import SaveToCsv
from Intelligent_well_control.src.models.LGBM import LGBModel
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score

In [4]:
class SaveToCsv():
    def __init__(self, csv_file = 'results1.csv'):
        self.csv_file = csv_file

    def save(self, result):
        fieldnames = list(result.keys())
        try:
            with open(self.csv_file, 'r') as file:
                # 文件已存在，不需要写表头
                pass
        except FileNotFoundError:
            with open(self.csv_file, 'w', newline='') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()

        # 打开CSV文件，追加新的结果
        with open(self.csv_file, 'a', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writerow(result)

        print('successfully save')
        return

In [6]:
SaveToCsv(r'\home\czl\project\Intelligent_well_control\reports\test_report.csv').save({'a' : 1})

successfully save


In [17]:

cur_data = pd.read_csv(r'E:\data\压井\新数据\间接数据\block_2_all.csv')

In [18]:
wells = cur_data['well_id'].unique().tolist()

In [15]:
wells

[7, 14, 32, 38, 40, 52, 55, 58, 2, 70, 78, 108]

In [16]:
train_well_ids = [[58, 40, 108], [58, 40, 108, 7, 78, 38], [58, 40, 108, 7, 78, 38, 52, 70, 2]]

In [27]:
for i in range(3):
    train_wells = train_well_ids[i]
    test_wells = [well_id for well_id in wells if well_id not in train_wells]
    print(train_wells, test_wells)
    
    labels = 'overflow_detected'
    rem_col_list = ['id', 'well_id', 'time', 'overflow_flag',
                    'work_state', 'invader_type', 'kill_main_method_x',
                    'deal_density', 'overflow_detected', 'block_id',
                    'standpipe_pressure', 'casing_pressure']
    feature_names = list(
                filter(lambda x: x not in rem_col_list, cur_data.columns))
    
    X_train = cur_data[cur_data['well_id'].isin(train_wells)][feature_names]
    Y_train = cur_data[cur_data['well_id'].isin(train_wells)][labels]
    X_test = cur_data[cur_data['well_id'].isin(test_wells)][feature_names]
    Y_test = cur_data[cur_data['well_id'].isin(test_wells)][labels]
    
    # 将数据转换为LightGBM的数据格式
    dtrain = lgb.Dataset(X_train, label=Y_train)
    dtest = lgb.Dataset(X_test, label=Y_test)
        
    # 设置XGBoost参数
    params = {
        'eta': 0.01,  # 学习率
        'metric': 'auc',
        'max_depth' : 10,
    }    
    
    # 训练模型
    model = lgb.train(params, dtrain, num_boost_round=500, valid_sets=[dtest])
    
    Y_pred = model.predict(X_test)
    print(Y_pred)
    print(Y_test)
    acc = accuracy_score(Y_pred, Y_test)
    print(acc)
    
    
    
    

[58, 40, 108] [7, 14, 32, 38, 52, 55, 2, 70, 78]
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017615 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9195
[LightGBM] [Info] Number of data points in the train set: 79855, number of used features: 235
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score 1.542195
[1]	valid_0's auc: 1
[2]	valid_0's auc: 1
[3]	valid_0's auc: 1
[4]	valid_0's auc: 1
[5]	valid_0's auc: 1
[6]	valid_0's auc: 1
[7]	valid_0's auc: 1
[8]	

ValueError: Classification metrics can't handle a mix of continuous and binary targets